# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
!pip install -U sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.3 MB/s eta 0:00:0000:01


# Homophone List

This needs to be expanded.

In [136]:
# Chat GPT Prompt: Give me a list of lists in python of 100 sets of homophones
homophones_list = [
    ["ate", "eight"],
    ["bare", "bear"],
    ["brake", "break"],
    ["capital", "capitol"],
    ["cell", "sell"],
    ["cite", "site", "sight"],
    ["complement", "compliment"],
    ["desert", "dessert"],
    ["die", "dye"],
    ["flour", "flower"],
    ["hear", "here"],
    ["hour", "our"],
    ["knight", "night"],
    ["know", "no"],
    ["mail", "male"],
    ["meat", "meet"],
    ["morning", "mourning"],
    ["one", "won"],
    ["pair", "pear"],
    ["peace", "piece"],
    ["principal", "principle"],
    ["rain", "reign", "rein"],
    ["right", "write"],
    ["sea", "see"],
    ["serial", "cereal"],
    ["sole", "soul"],
    ["stationary", "stationery"],
    ["tail", "tale"],
    ["threw", "through"],
    ["to", "too", "two"],
    ["weather", "whether"],
    ["week", "weak"],
    ["wear", "where"],
    ["which", "witch"],
    ["your", "you're"],
    ["allowed", "aloud"],
    ["board", "bored"],
    ["brake", "break"],
    ["capital", "capitol"],
    ["compliment", "complement"],
    ["desert", "dessert"],
    ["dual", "duel"],
    ["fair", "fare"],
    ["genre", "jinja"],
    ["hare", "hair"],
    ["here", "hear"],
    ["hoard", "horde"],
    ["loan", "lone"],
    ["pail", "pale"],
    ["peak", "peek", "pique"],
    ["profit", "prophet"],
    ["role", "roll"],
    ["root", "route"],
    ["sail", "sale"],
    ["scene", "seen"],
    ["serial", "cereal"],
    ["so", "sow"],
    ["stare", "stair"],
    ["steal", "steel"],
    ["their", "there", "they're"],
    ["throne", "thrown"],
    ["vain", "vein", "vane"],
    ["weak", "week"],
    ["wood", "would"],
    ["yew", "you"],
    ["bridal", "bridle"],
    ["cereal", "serial"],
    ["chord", "cord"],
    ["compliment", "complement"],
    ["dew", "due"],
    ["foul", "fowl"],
    ["grate", "great"],
    ["groan", "grown"],
    ["heal", "heel"],
    ["him", "hymn"],
    ["lay", "lie"],
    ["main", "mane"],
    ["marry", "merry"],
    ["mite", "might"],
    ["moose", "mousse"],
    ["mourn", "morn"],
    ["peace", "piece"],
    ["plum", "plumb"],
    ["pour", "pore"],
    ["rap", "wrap"],
    ["scene", "seen"],
    ["scent", "cent", "sent"],
    ["serial", "cereal"],
    ["shear", "sheer"],
    ["soar", "sore"],
    ["sow", "sew"],
    ["stake", "steak"],
    ["tide", "tied"],
    ["toe", "tow"],
    ["there", "their", "they're"],
    ["waist", "waste"],
    ["week", "weak"],
    ["write", "right", "rite"],
    ["won", "one"],
    ["your", "you're"],
    ["dear", "deer"],
]


# Load Model

Bert has been working decently well for me.

In [75]:
from transformers import pipeline

# Load the model and tokenizer
fill_mask = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Test case 

In [109]:
# Create input string
input_string = "Michigan one the football game against Ohio State!"

# Select homophone
target_homophone = "one"

# Get homophone options
homophone_options = [homophone for homophone in homophones_list if target_homophone in homophone]

# Replace homophone with mask token
input_string = input_string.replace(target_homophone, fill_mask.tokenizer.mask_token)

# Get results
results = fill_mask(input_string)

# Token string dict
token_string_dict = {}

# Get top results
for result in results:
   token_string_dict[result["token_str"]] = result["score"]

# Sort results
sorted_results = sorted(token_string_dict.items(), key=lambda x: x[1], reverse=True)

# Find top homophone in results
homophone_results = [result for result in sorted_results if result[0] in homophone_options[0]]

if homophone_results[0][0] == target_homophone:
    print("Correct!")
else:
    print("Incorrect!")
    print("Top result: ", homophone_results[0][0])
    print("Top result score: ", homophone_results[0][1])

Incorrect!
Top result:  won
Top result score:  0.704522430896759


## Building a function

In [186]:
# Function that can check a sentence that only contains one homophone
def one_homophone_checker(input_string, homophones_list=homophones_list, score_threshold=0):
    # Lowering the input string
    input_string = input_string.lower()

    # Flatten homophones list
    all_homophones = [
        word for homophone_set in homophones_list for word in homophone_set
    ]

    # Find homophones in input string
    target_homophone = list(set(input_string.split(" ")).intersection(set(all_homophones)))
    print(target_homophone)

    # If there are no homophones in the sentence, return the NA dataframe
    if len(target_homophone) < 1:
        output_df = pd.DataFrame(
            {
                "sentence": input_string,
                "has_homophone": False,
                "is_error": None,
                "error_idx": None,
                "error": None,
                "correct_word": None,
                "correct_sentence": None,
            }, index=[0]
        )
        return output_df

    else:
        target_homophone = target_homophone[0]

    # Get all homophone options from homophones list
    homophone_options = [homophone for homophone in homophones_list if target_homophone in homophone]

    # Replace homophone with mask token
    # CANNOT HANDLE MULTIPLE HOMOPHONES IN ONE SENTENCE
    masked_string = ' '.join([word if word != target_homophone else fill_mask.tokenizer.mask_token for word in input_string.split(" ")])
    print(masked_string)
    
    # Get results
    results = fill_mask(masked_string, top_k=100)

    # Token string dict
    token_string_dict = {}

    # Get top results and their score
    for result in results:
        try:
            token_string_dict[result["token_str"]] = result["score"]
        except TypeError:
            print(result)

    # Sort results
    sorted_results = sorted(token_string_dict.items(), key=lambda x: x[1], reverse=True)

    # Find top homophone in results
    homophone_results = [result for result in sorted_results if result[0] in homophone_options[0]]
    print(homophone_results)
    
    # If the top result is the target homophone, return the original sentence as it is correct
    if homophone_results[0][0] == target_homophone:
        final_sentence = input_string
        is_error = False
        correct_word = None
        error_idx = None
        error = None

    else:
        # If the top result is not the target homophone, check how many options it found
        # If multiple options:
        if len(homophone_results) > 1:

            # Check the difference between the top two results
            score_diff = homophone_results[0][1] - homophone_results[1][1]

            # If the difference is greater than the threshold, return the top result
            if score_diff > score_threshold:
                error_idx = input_string.split(" ").index(target_homophone)
                error = target_homophone
                final_sentence = input_string.replace(target_homophone, homophone_results[0][0])
                correct_word = homophone_results[0][0]
                is_error = True
        
        # If the difference is less than the threshold, return the original sentence
        else:
            is_error = True
            error_idx = input_string.split(" ").index(target_homophone)
            final_sentence = input_string.replace(target_homophone, homophone_results[0][0])
            correct_word = homophone_results[0][0]
            error = target_homophone

    # Create output dataframe that matches our test data
    output_df = pd.DataFrame(
            {
                "sentence": input_string,
                "has_homophone": True,
                "is_error": is_error,
                "error_idx": error_idx,
                "error": error,
                "correct_word": correct_word,
                "correct_sentence": final_sentence,
            }, index=[0]
        )
    return output_df

## Testing

In [187]:
input_string = "At lunch today, I had way to much food!"

one_homophone_checker(input_string)

['to']
at lunch today, i had way [MASK] much food!
[('too', 0.9997673630714417), ('to', 4.6690133785887156e-06)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,"at lunch today, i had way to much food!",True,True,6,to,too,"at lunch tooday, i had way too much food!"


In [188]:
input_string = "Michigan one the game against Ohio State!"

one_homophone_checker(input_string)

['one']
michigan [MASK] the game against ohio state!
[('won', 0.7407358884811401)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,michigan one the game against ohio state!,True,True,1,one,won,michigan won the game against ohio state!


# NEXT STEPS: MAKE SENTENCES LIKE THIS WORK WITH MULTIPLE ERRORS

In [191]:
input_string = "Have you scene my new waist of money?"

one_homophone_checker(input_string)

['waist', 'scene', 'you']
have you scene my new [MASK] of money?
[('waste', 0.003929820377379656)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,have you scene my new waist of money?,True,True,5,waist,waste,have you scene my new waste of money?
